In [1]:
import os
import getpass
from dotenv import load_dotenv
from tavily import TavilyClient

# Load environment variables from .env file
load_dotenv()

# Prompt the user to securely input the API key if not already set in the environment
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

# Initialize the Tavily API client using the loaded or provided API key
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

In [2]:
# Define the set of web tools our agent will use to interact with the Tavily API.
from langchain_tavily import TavilySearch
from langchain_tavily import TavilyExtract
from langchain_tavily import TavilyCrawl

# Define the LangChain search tool
search = TavilySearch(max_results=10, topic="general")

# Define the LangChain extract tool
extract = TavilyExtract(extract_depth="advanced")

# Define the LangChain crawl tool
crawl = TavilyCrawl()

In [3]:
# Instantiate the OpenAI foundation models
from langchain_openai import ChatOpenAI

# o3-mini-2025-01-31
o3_mini = ChatOpenAI(model="o3-mini-2025-01-31", api_key=os.getenv("OPENAI_API_KEY"))

# gpt-4.1
gpt_4_1 = ChatOpenAI(model="gpt-4.1", api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
import datetime
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

today = datetime.datetime.today().strftime("%A, %B %d, %Y")

# Create the web agent
web_agent = create_react_agent(
    model=gpt_4_1,
    tools=[search, extract, crawl],
    prompt=ChatPromptTemplate.from_messages(
        [
            (
                "system",
                f"""    
        You are a research agent equipped with advanced web tools: Tavily Web Search, Web Crawl, and Web Extract. Your mission is to conduct comprehensive, accurate, and up-to-date research, grounding your findings in credible web sources.

        **Today's Date:** {today}

        **Available Tools:**

        1. **Tavily Web Search**

        * **Purpose:** Retrieve relevant web pages based on a query.
        * **Usage:** Provide a search query to receive semantically ranked results, each containing the title, URL, and a content snippet.
        * **Best Practices:**

            * Use specific queries to narrow down results.
            * Optimize searches using parameters such as `search_depth`, `time_range`, `include_domains`, and `include_raw_content`.
            * Break down complex queries into specific, focused sub-queries.

        2. **Tavily Web Crawl**

        * **Purpose:** Explore a website's structure and gather content from linked pages for deep research and information discovery from a single source.
        * **Usage:** Input a base URL to crawl, specifying parameters such as `max_depth`, `max_breadth`, and `extract_depth`.
        * **Best Practices:**

            * Begin with shallow crawls and progressively increase depth.
            * Utilize `select_paths` or `exclude_paths` to focus the crawl.
            * Set `extract_depth` to "advanced" for comprehensive extraction.

        3. **Tavily Web Extract**

        * **Purpose:** Extract the full content from specific web pages.
        * **Usage:** Provide URLs to retrieve detailed content.
        * **Best Practices:**

            * Set `extract_depth` to "advanced" for detailed content, including tables and embedded media.
            * Enable `include_images` if image data is necessary.

        **Guidelines for Conducting Research:**

        * **Citations:** Always support findings with source URLs, clearly provided as in-text citations.
        * **Accuracy:** Rely solely on data obtained via provided tools—never fabricate information.
        * **Methodology:** Follow a structured approach:

        * **Thought:** Consider necessary information and next steps.
        * **Action:** Select and execute appropriate tools.
        * **Observation:** Analyze obtained results.
        * Repeat Thought/Action/Observation cycles as needed.
        * **Final Answer:** Synthesize and present findings with citations in markdown format.

        **Example Workflows:**

        **Workflow 1: Search Only**

        **Question:** What are recent news headlines about artificial intelligence?

        * **Thought:** I need quick, recent articles about AI.
        * **Action:** Use Tavily Web Search with the query "recent artificial intelligence news" and set `time_range` to "week".
        * **Observation:** Retrieved 10 relevant articles from reputable news sources.
        * **Final Answer:** Summarize key headlines with citations.

        **Workflow 2: Search and Extract**

        **Question:** Provide detailed insights into recent advancements in quantum computing.

        * **Thought:** I should find recent detailed articles first.
        * **Action:** Use Tavily Web Search with the query "recent advancements in quantum computing" and set `time_range` to "month".
        * **Observation:** Retrieved 10 relevant results.
        * **Thought:** I should extract content from the most comprehensive article.
        * **Action:** Use Tavily Web Extract on the most relevant URL from search results.
        * **Observation:** Extracted detailed information about quantum computing advancements.
        * **Final Answer:** Provide detailed insights summarized from extracted content with citations.

        **Workflow 3: Search and Crawl**

        **Question:** What are the latest advancements in renewable energy technologies?

        * **Thought:** I need recent articles about advancements in renewable energy.
        * **Action:** Use Tavily Web Search with the query "latest advancements in renewable energy technologies" and set `time_range` to "month".
        * **Observation:** Retrieved 10 articles discussing recent developments in solar panels, wind turbines, and energy storage.
        * **Thought:** To gain deeper insights, I'll crawl a relevant industry-leading renewable energy site.
        * **Action:** Use Tavily Web Crawl on the URL of a leading renewable energy industry website, setting `max_depth` to 2.
        * **Observation:** Gathered extensive content from multiple articles linked on the site, highlighting new technologies and innovations.
        * **Final Answer:** Provide a synthesized summary of findings with citations.

        ---

        You will now receive a research question from the user:

        """,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    ),
    name="web_agent",
)

In [7]:
from langchain.schema import HumanMessage

# Test the web agent
inputs = {
    "messages": [
        HumanMessage(
            content="find all FIFA club world cup matches remaining, also finn the strongest contender for the club world cup"
        )
    ]
}

# Stream the web agent's response
for s in web_agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

find all FIFA club world cup matches remaining, also finn the strongest contender for the club world cup
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  tavily_search (call_VVncwYch1KdjBmJcFvoI8keS)
 Call ID: call_VVncwYch1KdjBmJcFvoI8keS
  Args:
    query: FIFA Club World Cup 2025 remaining matches schedule
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "FIFA Club World Cup 2025 remaining matches schedule", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "FIFA Club World Cup 2025 Schedule: Full Match Fixtures (PDF)", "url": "https://worldcuppass.com/fifa-club-world-cup-2025-schedule/", "content": "| DATE | MATCH | TEAM | TIME (ET) | VENUE |\n| --- | --- | --- | --- | --- |\n| Sat, 28 Jun | Match 49 | Winner Group A vs R

In [8]:
from IPython.display import Markdown

Markdown(message.content)

Thought:
- I have found the latest and official sources for the remaining matches and schedule of the FIFA Club World Cup 2025.
- I have also found betting odds and expert analysis identifying the strongest contenders for the tournament.

Final Answer:

## Remaining FIFA Club World Cup 2025 Matches

The knockout and final matches take place starting from June 28, 2025, with the schedule as follows (all times ET):

- **Saturday, June 28**
  - Match 49: Winner Group A vs Runner-up Group B, 12pm, Lincoln Financial Field (Philadelphia)
  - Match 50: Winner Group C vs Runner-up Group D, 4pm, Bank of America Stadium (Charlotte)
- **Sunday, June 29**
  - Match 51: Winner Group B vs Runner-up Group A, 12pm, Mercedes-Benz Stadium (Atlanta)
  - Match 52: Winner Group D vs Runner-up Group C, 4pm, Hard Rock Stadium (Miami)
- **Monday, June 30**
  - Match 53: Winner Group E vs Runner-up Group F, 3pm, Bank of America Stadium (Charlotte)
  - Match 54: Winner Group G vs Runner-up Group H, 9pm, Camping World Stadium (Orlando)
- **Tuesday, July 1**
  - Match 55: Winner Group H vs Runner-up Group G, 3pm, Hard Rock Stadium (Miami)
  - Match 56: Winner Group F vs Runner-up Group E, 9pm, Mercedes-Benz Stadium (Atlanta)
- Further rounds (Quarterfinals, Semi-finals, Final) will follow until the trophy match on July 13, 2025 at MetLife Stadium, New Jersey.

Sources: 
- [World Cup Pass](https://worldcuppass.com/fifa-club-world-cup-2025-schedule/)
- [NBC Sports Club World Cup 2025 Schedule](https://www.nbcsports.com/soccer/news/2025-fifa-club-world-cup-schedule-fixtures-date-teams-host-cities)
- [Wikipedia, 2025 FIFA Club World Cup](https://en.wikipedia.org/wiki/2025_FIFA_Club_World_Cup)

---

## Strongest Contenders for FIFA Club World Cup 2025

Based on latest bookmaker odds and expert analysis, the top contenders and favorites are:

- **Real Madrid** (UEFA): Odds range 4/1 (~+400)
- **Paris Saint-Germain (PSG)** (UEFA): Odds range 5/1 (2nd favorite)
- **Manchester City** (UEFA): Odds range 5/1
- **Bayern Munich** (UEFA): Odds range 7/1

These UEFA giants dominate predictions due to their recent form, squad strength, and performances in continental competitions. Other teams with outside chances include Chelsea, Inter Milan, and Flamengo, but the heavy favorites according to all major sportsbooks and football analysts, are Real Madrid, PSG, and Manchester City.

Sources:
- [Oddspedia Club World Cup Odds](https://oddspedia.com/us/insights/soccer/fifa-club-world-cup-odds)
- [Sporting News Betting Favourites](https://www.sportingnews.com/uk/betting/news/club-world-cup-2025-betting-odds-favourites-win-prediction/68b3942f4ad7ee5dc18e19e1)
- [SI.com Club World Cup Favorites](https://www.si.com/soccer/fifa-club-world-cup-2025-favorites-top-contenders)
- [FanDuel Power Rankings](https://www.fanduel.com/research/what-are-the-best-teams-in-the-2025-fifa-club-world-cup-power-ranking-all-32-teams)

**Summary:** The remaining Club World Cup matches will be played through to July 13, 2025, with Real Madrid, PSG, and Manchester City widely seen as the strongest favorites to win the title.

In [9]:
# Test the web agent
inputs = {
    "messages": [
        HumanMessage(
            content="return 5 job postings for a software engineer in the bay area on linkedin"
        )
    ]
}

# Stream the web agent's response
for s in web_agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

return 5 job postings for a software engineer in the bay area on linkedin
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  tavily_search (call_YIIYBUwLWjwl6cKxvw9avhQz)
 Call ID: call_YIIYBUwLWjwl6cKxvw9avhQz
  Args:
    query: software engineer job postings Bay Area site:linkedin.com
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "software engineer job postings Bay Area site:linkedin.com", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.linkedin.com/jobs/search/?geoId=90000084&keywords=software%20engineer&location=San%20Francisco%20Bay%20Area", "title": "135000+ Software Engineer jobs in San Francisco Bay Area", "content": "Get notified about new Software Engineer jobs in San Francisco Bay Area. Sign in to cre

In [11]:
# Print the agent's final response as markdown
Markdown(message.content)

Here are 5 recent software engineer job postings in the Bay Area from LinkedIn:

1. **Software Engineer, Backend**  
   - **Company:** Tinder  
   - **Location:** Palo Alto, CA  
   - **Posted:** 12 hours ago  
   - [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-backend-at-tinder-4257030455)

2. **Software Engineer, AI Intern (Fall 2025)**  
   - **Company:** Notion  
   - **Location:** San Francisco Bay Area  
   - **Posted:** 1 week ago  
   - [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-ai-intern-fall-2025-at-notion-4252384719)

3. **Software Engineer, Growth**  
   - **Company:** Notion  
   - **Location:** San Francisco, CA  
   - **Posted:** 1 week ago  
   - [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-growth-at-notion-4143892275)

4. **Software Engineer, Developer Experience**  
   - **Company:** Notion  
   - **Location:** San Francisco Bay Area  
   - **Posted:** 1 week ago  
   - [View Job Posting](https://www.linkedin.com/jobs/view/software-engineer-developer-experience-at-notion-4136588579)

5. **Full-Stack Engineer Intern**  
   - **Company:** Harmony  
   - **Location:** San Francisco Bay Area  
   - **Posted:** 1 week ago  
   - [View Job Posting](https://www.linkedin.com/jobs/view/full-stack-engineer-intern-at-harmony-4253794938)

Source: [LinkedIn job search for Software Engineers in the Bay Area](https://www.linkedin.com/jobs/search/?geoId=90000084&keywords=software%20engineer&location=San%20Francisco%20Bay%20Area)